In [1]:
# 이것저것 import
import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from collections import Counter
from wordcloud.wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# 구글 드라이브와 내 서버 연결 및 작업디렉토리 변경
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/KDT/세미프로젝트'

Mounted at /content/drive
/content/drive/MyDrive/KDT/세미프로젝트


In [3]:
df = pd.read_csv('kr3_raw.tsv', sep = '\t')
df

,Region,Rating,Category,Review
0,인덕원,1,삼겹살 고기집,숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편...
1,인덕원,1,삼겹살 고기집,고기가 정말 맛있었어요! 육즙이 가득있어서 너무 좋았아요\n일하시는분들 너무 친절하...
2,인덕원,1,순대국 찹쌀순대,"잡내없고 깔끔, 담백한 맛의 순대국이 순대국을 안 좋아하는 사람들에게도 술술 넘어갈..."
3,인덕원,1,순대국 찹쌀순대,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다~~ 맛도 좋습니다\n다만 양념...
4,인덕원,1,순대국 찹쌀순대,순대국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요.\n\n그러나 ...
...,...,...,...,...
642408,평택,2,일식/중식/세계음식,"요즘, 핫하게,,,떠오르구 있는 중국집. ㅋ, 맥주의 여파루 속이 안좋지만 와봄. ..."
642409,평택,0,한식,원래 글 안쓰는데 이거는 정말 다른분들 위해서 써야할것같네요 방금 포장주문 해서 왔...
642410,평택,1,한식,"우리팀 단골집, 술먹고 다음 날 가면 푸짐하게 배불리 해장 할 수 있는곳, 주말도 ..."
642411,평택,2,카페/디저트,"원래는 평택에 있었는데, 연남동에도 최근에 생겨서 방문 했는데..진짜 줄이 어마어마..."


In [4]:
# 누락 데이터 있나요~
df.isnull().sum()

Region       0
Rating       0
Category    24
Review       0
dtype: int64

In [5]:
# 제거
data = df.dropna()
data.isnull().sum()

Region      0
Rating      0
Category    0
Review      0
dtype: int64

In [6]:
positive = data.loc[(data['Rating'] == 1) & (data['Category'].isin(['한식', '기타 한식', '삼겹살','고기/구이류', '고기 요리', '탕/찌개/전골','한정식/백반/전통 한식','갈비','한정식', '','냉면'])), ['Review']].reset_index(drop=True)
negative = data.loc[(data['Rating'] == 0) & (data['Category'].isin(['한식', '기타 한식', '삼겹살','고기/구이류', '고기 요리', '탕/찌개/전골','한정식/백반/전통 한식','갈비','한정식', '','냉면'])), ['Review']].reset_index(drop=True)

In [7]:
print(positive)
print('='*100)
print(negative)

                                                  Review
0            개인적으로 제일 좋아하는 냉면집\n진짜 집 근처였으면 일주일에 한번씩갈것같아요
1      방화동 구석의 정말 허름한 노포. 아저씨의 응대가 유쾌하고 살갑다. 손으로 쓴 메뉴...
2           아주머니 혼자하시는 식당이고 푸짐해요\n국물이 사골이라 맛있어요 순대는 쏘쏘해요
3      장어가 담백하면서 고소하고 맛있습니다. 비린맛이 적어 물리지 않고 맛있게 먹을 수 ...
4             오픈 이벤트로 막국수 할인 중이며, 맛있고 푸짐하여 가족 외식으로 괜찮은 곳
...                                                  ...
50137  가격대비 괜찮긴한데 가격을 자꾸 올리고 샐러드바 메뉴는 줄어드는 경향이 있어서 살짝...
50138  리뷰에직원이 친절하지않다더니 정말이네요 선불이라 만원짜리내고 기다리고있는데 거지동냥...
50139  송탄 미군부대 근처에 위치한 곳 원래 로컬맛집으로 되게 유명했는데 삼대천왕에 나온 ...
50140  집에서 40키로 정도 떨어져 있는 곳인데도 몇달에 한번은 이거 먹으러 일부러 갑니다...
50141  우리팀 단골집, 술먹고 다음 날 가면 푸짐하게 배불리 해장 할 수 있는곳, 주말도 ...

[50142 rows x 1 columns]
                                                  Review
0      맛있었던 기억을 갖고 오랜만에 찾았는데, 반찬도 불고기도 보통수준으로 전락??? 하...
1                   한번 먹은 걸로 만족 했어요\n나쁘지는 안아요\n집에서 드시는걸로
2      아~ 인터넷 후기 보고 서울에서 목포까지 5시간 넘게 운전해서 갔다가 폭 망한 집입...
3      1인분 11,000원의 정식을 먹었는데 평범합니다. 개인적으로 보리밥은 그냥 비빔밥...
4    

In [8]:
def preprocess(data):
  my_split = []
  for pas in data.iloc:
    data = (pas['Review'])
    split_data = re.sub('[#"+*!.,]', '', data).split(' ')
    for i in split_data:
      if len(i) >= 2:
        my_split.append(i)
  return my_split

In [25]:
# pos = preprocess(positive)
# neg = preprocess(negative)

# 쓸모 없는 단어 빼기
good_remove = {'너무', '정말', '진짜', '맛있게', '많이', '있는', '먹고', '맛이', '맛있고', '맛있는'}
bad_remove = {'너무', '그냥', '드가자~', '정말', '맛은', '맛이', '진짜', '많이', '맛도', '고기', '먹고', '다른', '갔는데', '별로', '다시', '없고', '하고', '여기',
              '그리고', '있는', '절대', '엄청', '다시는', '먹을', '이렇게', '없음', '이런', '먹었는데', '해서', '가서', '않고', '완전', '아니고', '다신', '먹는',
              '손님', '양이', '기분', '계속', '최악', '보고', '근데', '고기는', '고기가', '시켰는데', '하는', '나오는', '없는', '손님이', '처음', '식당', '내가',
              'ㅠㅠ', '음식이', '고기를', '그렇게', '했는데', '없이', '가격은', '가격이'}

my_pos = [i for i in pos if i not in good_remove]
my_neg = [i for i in neg if i not in bad_remove]


# 등장 횟수 카운팅
good_counts = Counter(my_pos)
bad_counts = Counter(my_neg)

# 상위 100단어
print(good_counts.most_common(100))
print(bad_counts.most_common(100))

[('맛있어요', 3953), ('고기', 2717), ('같이', 2515), ('먹을', 2313), ('좋은', 2190), ('다른', 2153), ('엄청', 2113), ('아주', 2089), ('그냥', 2080), ('조금', 2031), ('여기', 2025), ('먹으면', 1879), ('먹는', 1852), ('그리고', 1838), ('먹었는데', 1790), ('좋고', 1748), ('ㅎㅎ', 1740), ('맛도', 1728), ('맛있다', 1715), ('맛은', 1639), ('않고', 1631), ('있어서', 1613), ('나오는', 1589), ('맛집', 1575), ('맛있었어요', 1571), ('양이', 1502), ('고기가', 1496), ('완전', 1457), ('고기를', 1438), ('깔끔하고', 1433), ('맛있어서', 1410), ('좋아요', 1353), ('함께', 1316), ('양도', 1310), ('제일', 1301), ('같아요', 1261), ('특히', 1258), ('역시', 1254), ('바로', 1241), ('맛을', 1241), ('맛있음', 1225), ('모두', 1204), ('직접', 1198), ('살짝', 1176), ('가장', 1163), ('그래도', 1131), ('고기도', 1128), ('떡볶이', 1128), ('약간', 1119), ('계속', 1107), ('하지만', 1102), ('같은', 1101), ('이렇게', 1100), ('다시', 1051), ('처음', 1045), ('항상', 1034), ('좋았어요', 1026), ('저는', 1020), ('많아서', 1018), ('없는', 1018), ('많고', 1016), ('매우', 1003), ('자주', 1002), ('갔는데', 1001), ('가격이', 994), ('근데', 961), ('부드럽고', 961), ('하는', 959), ('최고', 953), ('유명한